In [4]:
import numpy as np
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend
from ndlib.viz.bokeh.DiffusionPrevalence import DiffusionPrevalence
from bokeh.io  import output_notebook, show
from bokeh.plotting import figure, from_networkx

output_notebook()

Loading BokehJS ...

## Base SIR model

In [5]:
# Seed
rnd_seed = 42

# Network topology
g = nx.erdos_renyi_graph(1000, 0.1, seed=rnd_seed)

# Model selection
model = ep.SIRModel(g)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.001)
cfg.add_model_parameter('gamma', 0.01)
cfg.add_model_parameter("fraction_infected", 0.01)
model.set_initial_status(cfg)


# Simulation execution
iterations = model.iteration_bunch(1000)
trends = model.build_trends(iterations)

In [6]:
# Visualization
viz = DiffusionTrend(model, trends)
p = viz.plot(width=800, height=600)
show(p)

## Darwinian model

In [113]:
import ndlib.models.CompositeModel as gc
import ndlib.models.ModelConfig as mc
import ndlib.models.compartments.NodeStochastic as NodeStochastic
import ndlib.models.compartments.NodeCategoricalAttribute as NodeCategoricalAttribute
import ndlib.models.compartments.NodeNumericalAttribute as NodeNumericalAttribute
import ndlib.models.compartments.NodeThreshold as NodeThreshold
import ndlib.models.compartments.CountDown as CountDown
import ndlib.models.compartments.ConditionalComposition as ConditionalComposition

In [181]:
# Disease parameters
population_size = 1000
rndgraph_proba_edge = 0.15
t_steps = 500
infected_ratio_0 = 0.01
pay_period = 30  # try 15
out_time = 3

# SIR parameters
S_to_I = 0.2
I_to_R = 0.01
I_to_D = 0.001

# France age distribution: https://www.insee.fr/fr/statistiques/2381474
age_windows = [[0,14], [15,19], [20,24], [25,29], [30,34], [35,39], [40,44], [45,49], [50,54], [55,59], [60,64], [65,69], [70,74], [75, 99]]
hist_M = [0.188256545, 0.065574938, 0.058721502, 0.056633943, 0.060702322, 0.063561676, 0.061788497, 0.068895165, 0.067157668, 0.065402145, 0.06033513, 0.056717716, 0.049711797, 0.076540954]
hist_W = [0.168599713, 0.058170037, 0.053511884, 0.054191935, 0.060285392, 0.062670518, 0.059724852, 0.065775675, 0.064904431, 0.064630968, 0.061874505, 0.059493995, 0.05384376, 0.112322337]
hist_age = [hist_M, hist_W]

In [182]:
# Starting network
g = nx.erdos_renyi_graph(1000, 0.1, seed=rnd_seed)
true_event = NodeStochastic(1.0)

# Node attributes: age, base energy, base resources
attr = {}
for n in g.nodes():
    n_sex = np.random.choice([0, 1])
    n_agegroup = np.random.choice(range(len(age_windows)), p=hist_age[n_sex])
    n_age = np.random.choice(age_windows[n_agegroup])
    n_energy = np.random.uniform(5.0, 20.0)
    attr[n] = {"Sex": n_sex, "Age": n_age, "Energy": n_energy, "Money": 30.0, "Buys": "n", "Hunger": "n", "buy_time": out_time}

nx.set_node_attributes(g, attr)

# Composite Model instantiation
model = gc.CompositeModel(g)

# Model stata
model.add_status("Susceptible")
model.add_status("Infected")
model.add_status("Removed")
model.add_status("Dead")

# Model rules
## As rules are sequentially evaluated, going out is before applying diffusion rules

# - Economical
##  If an individual decides to buy, he's Susceptible
##  otherwise he's confined

# Die of hunger
model.add_rule("Susceptible", "Dead", NodeCategoricalAttribute("Hunger", "y"))
model.add_rule("Infected", "Dead", NodeCategoricalAttribute("Hunger", "y"))

#  - Epidemiological
model.add_rule("Susceptible", "Infected", NodeStochastic(S_to_I, triggering_status="Infected", composed=NodeCategoricalAttribute("Buys", "y")))
model.add_rule("Infected", "Removed", NodeStochastic(I_to_R))
model.add_rule("Infected", "Dead", NodeStochastic(I_to_D))





In [183]:
# Model initial status configuration
config = mc.Configuration()
config.add_model_parameter('fraction_infected', infected_ratio_0)

model.set_initial_status(config)

In [184]:
# Simulation execution
iterations = []
for it in range(0, t_steps):
#     At each timestep, decrease energy to encourage consumption
    for n in g.nodes():
#     Payday
        if it % pay_period == 0:
            attr[n]["Money"] += 10.0
#     Base energy consumption
        attr[n]["Energy"] -= 1.0
    
#     Buy if low energy
        if np.random.random() < np.exp(attr[n]["Energy"]) and attr[n]["Money"] >= 1.0:
            attr[n]["Buys"] = "y"
            attr[n]["Energy"] += 5.0
            attr[n]["Money"] -= 1.0
            
            
#     Mark hunger death
        elif attr[n]["Energy"] <= 0.0:
            attr[n]["Hunger"] = "y"
            
#     Time exposed
        if attr[n]["buy_time"] == 0:
            attr[n]["buy_time"] = out_time
            attr[n]["Buys"] = "n"
            
        if attr[n]["Buys"] == "y":
            attr[n]["buy_time"] -= 1
            
    nx.set_node_attributes(g, attr)
    
#     Run Epi-model
    its = model.iteration(node_status=True)
#     status_t = its['status']
#     update_on_beds(model, status_t, crit_dead_id)
    
    iterations.append(its)

In [185]:
trends = model.build_trends(iterations[:200])

In [186]:
# Visualization
viz = DiffusionTrend(model, trends)
p = viz.plot(width=800, height=600)
show(p)

### Baseline

In [82]:
# Starting network
g = nx.erdos_renyi_graph(1000, 0.1, seed=rnd_seed)
true_event = NodeStochastic(1.0)

# Composite Model instantiation
model = gc.CompositeModel(g)

# Model stata
model.add_status("Susceptible")
model.add_status("Infected")
model.add_status("Removed")
model.add_status("Dead")

# Model rules
## As rules are sequentially evaluated, going out is before applying diffusion rules

# - Economical
##  If an individual decides to buy, he's Susceptible
##  otherwise he's confined

# Die of hunger
# model.add_rule("Susceptible", "Dead", NodeCategoricalAttribute("Hunger", "y"))
# model.add_rule("Infected", "Dead", NodeCategoricalAttribute("Hunger", "y"))

#  - Epidemiological
model.add_rule("Susceptible", "Infected", NodeStochastic(S_to_I, triggering_status="Infected"))
model.add_rule("Infected", "Removed", NodeStochastic(I_to_R))
model.add_rule("Infected", "Dead", NodeStochastic(I_to_D))

# Model initial status configuration
config = mc.Configuration()
config.add_model_parameter('fraction_infected', infected_ratio_0)

model.set_initial_status(config)

In [83]:
# Simulation execution
iterations = model.iteration_bunch(t_steps)

In [151]:
trends = model.build_trends(iterations[:200])
# Visualization
viz = DiffusionTrend(model, trends)
p = viz.plot(width=800, height=600)
show(p)